In [1]:
import joblib
import json

from IPython.display import display

import numpy as np
import pandas as pd
from sqlalchemy import create_engine, text

from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import GridSearchCV

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [2]:
with open('../constants.json') as CONSTANTS_FILE:
    CONSTANTS = json.load(CONSTANTS_FILE)
    
DB_ENGINE = create_engine(f'postgresql+psycopg2://{CONSTANTS["DB_USER"]}:{CONSTANTS["DB_PASSWORD"]}@{CONSTANTS["DB_IP"]}:{CONSTANTS["DB_PORT"]}/{CONSTANTS["DB_NAME"]}', future=True)
CONN = DB_ENGINE.connect()

In [3]:
DATA = pd.read_sql(text('SELECT * FROM wines'), CONN)
Y_COLUMN_NAME = 'price'

In [4]:
DATA

,id,name,is_red,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol,quality,price
0,0,White Claret,0,7.0,0.17,0.74,12.80,0.045,24.0,126.0,0.99420,3.26,0.38,12.2,8,629.79
1,1,Red Boal or Bual,1,7.7,0.64,0.21,2.20,0.077,32.0,133.0,0.99560,3.27,0.45,9.9,5,665.47
2,2,White Fumé Blanc,0,6.8,0.39,0.34,7.40,0.020,38.0,133.0,0.99212,3.18,0.44,12.0,7,483.00
3,3,White Trebbiano,0,6.3,0.28,0.47,11.20,0.040,61.0,183.0,0.99592,3.12,0.51,9.5,6,440.72
4,4,White Pinot Blanc,0,7.4,0.35,0.20,13.90,0.054,63.0,229.0,0.99888,3.11,0.50,8.9,6,458.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6492,White Sherry,0,7.6,0.32,0.58,16.75,0.050,43.0,163.0,0.99990,3.15,0.54,9.2,5,394.45
6493,6493,White Sauterns,0,5.6,0.28,0.27,3.90,0.043,52.0,158.0,0.99202,3.35,0.44,10.7,7,480.52
6494,6494,White Tokay,0,6.4,0.37,0.20,5.60,0.117,61.0,183.0,0.99459,3.24,0.43,9.5,5,339.10
6495,6495,White Müller-Thurgau,0,6.5,0.26,0.50,8.00,0.051,46.0,197.0,0.99536,3.18,0.47,9.5,5,397.43


In [5]:
RAW_NUMERICAL_COLUMN_NAMES = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'ph', 'sulphates', 'alcohol']
RAW_CATEGORICAL_COLUMN_NAMES = ['is_red']

In [6]:
%%time
print("Correlacao de cada coluna numerica com Y\n")
to_use_numerical_columns = []

if True:
    TRANFORMER = Normalizer()
    NUM_COLUMNS = TRANFORMER.fit_transform(DATA[RAW_NUMERICAL_COLUMN_NAMES])
    CORR = pd.concat([pd.DataFrame(NUM_COLUMNS, columns=RAW_NUMERICAL_COLUMN_NAMES), DATA[Y_COLUMN_NAME]], axis=1).corr()[Y_COLUMN_NAME].abs().sort_values(ascending=False)
    print(CORR[CORR>0.05])
    to_use_numerical_columns = CORR[CORR>0.05].index.to_list()
    to_use_numerical_columns.remove(Y_COLUMN_NAME)
    print()
    print(f'Total = {len(to_use_numerical_columns)} colunas numericas')
    print()

Correlacao de cada coluna numerica com Y

price                   1.000000
sulphates               0.677846
alcohol                 0.630777
fixed_acidity           0.627671
ph                      0.622581
density                 0.619801
total_sulfur_dioxide    0.591939
chlorides               0.559533
volatile_acidity        0.536301
citric_acid             0.466180
free_sulfur_dioxide     0.339917
residual_sugar          0.184567
Name: price, dtype: float64

Total = 11 colunas numericas

CPU times: total: 0 ns
Wall time: 14 ms


In [7]:
%%time
print("Correlacao de cada coluna categorica com Y\n")
to_use_categorical_columns = []

if True:
    TRANFORMER = OneHotEncoder(drop='first', sparse=False)
    CAT_COLUMNS = TRANFORMER.fit_transform(DATA[RAW_CATEGORICAL_COLUMN_NAMES])
    CORR = pd.concat([pd.DataFrame(CAT_COLUMNS, columns=RAW_CATEGORICAL_COLUMN_NAMES), DATA[Y_COLUMN_NAME]], axis=1).corr()[Y_COLUMN_NAME].abs().sort_values(ascending=False)
    print(CORR[CORR>0.05])
    to_use_categorical_columns = CORR[CORR>0.05].index.to_list()
    to_use_categorical_columns.remove(Y_COLUMN_NAME)
    print()
    print(f'Total = {len(to_use_categorical_columns)} colunas categoricas')
    print()

Correlacao de cada coluna categorica com Y

price     1.000000
is_red    0.806658
Name: price, dtype: float64

Total = 1 colunas categoricas

CPU times: total: 0 ns
Wall time: 7 ms


In [8]:
to_use_numerical_column_numbers = []
for COLUMN_NAME in to_use_numerical_columns:
    to_use_numerical_column_numbers.append(DATA.columns.to_list().index(COLUMN_NAME))

In [9]:
to_use_categorical_column_numbers = []
for COLUMN_NAME in to_use_categorical_columns:
    to_use_categorical_column_numbers.append(DATA.columns.to_list().index(COLUMN_NAME))

In [10]:
PRE_PROCESSOR =  ColumnTransformer([('normalize', Normalizer(), to_use_numerical_column_numbers),
                                 ('pass', OneHotEncoder(), to_use_categorical_column_numbers)])

In [11]:
DATA_X = PRE_PROCESSOR.fit_transform(DATA.to_numpy())
DATA_Y = DATA['price'].to_numpy()

In [12]:
%%time
if True:
    HP_TUNNER = GridSearchCV(LinearRegression(), {'fit_intercept':[False, True]}, n_jobs=-1)
    HP_TUNNER.fit(DATA_X, DATA_Y)
    
    print('---LinearRegression---')
    print(f'Best params = {HP_TUNNER.best_params_}')
    print()
    display(pd.DataFrame(HP_TUNNER.cv_results_).sort_values(by='rank_test_score', ignore_index=True).head())
    print()

---LinearRegression---
Best params = {'fit_intercept': True}



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_fit_intercept,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005601,0.001200,0.0010,0.000002,True,{'fit_intercept': True},0.716651,0.733099,0.728404,0.752872,0.706542,0.727514,0.015702,1
1,0.008400,0.005425,0.0014,0.001021,False,{'fit_intercept': False},0.716651,0.733099,0.728361,0.752900,0.706542,0.727511,0.015711,2



CPU times: total: 62.5 ms
Wall time: 2.09 s


In [13]:
%%time
if True:
    HP_TUNNER = GridSearchCV(DecisionTreeRegressor(random_state=RANDOM_SEED), {'max_depth':[4, 5, 6], 'criterion':['squared_error', 'friedman_mse', 'absolute_error'], 'splitter':['best']}, n_jobs=-1)
    HP_TUNNER.fit(DATA_X, DATA_Y)
    
    print('---DecisionTreeClassifier---')
    print(f'Best params = {HP_TUNNER.best_params_}')
    print()
    display(pd.DataFrame(HP_TUNNER.cv_results_).sort_values(by='rank_test_score', ignore_index=True).head())
    print()

---DecisionTreeClassifier---
Best params = {'criterion': 'absolute_error', 'max_depth': 5, 'splitter': 'best'}



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,2.848801,0.203800,0.001002,1.248913e-06,absolute_error,5,best,"{'criterion': 'absolute_error', 'max_depth': 5...",0.687578,0.726007,0.704513,0.708551,0.685732,0.702476,0.014813,1
1,0.062197,0.005844,0.001001,8.662177e-07,friedman_mse,6,best,"{'criterion': 'friedman_mse', 'max_depth': 6, ...",0.697512,0.723205,0.691205,0.702407,0.683149,0.699496,0.013494,2
2,0.060798,0.004119,0.002002,1.551083e-03,squared_error,6,best,"{'criterion': 'squared_error', 'max_depth': 6,...",0.697512,0.723205,0.688210,0.702407,0.683149,0.698897,0.013909,3
3,0.054798,0.008423,0.001401,4.897079e-04,squared_error,5,best,"{'criterion': 'squared_error', 'max_depth': 5,...",0.685056,0.722808,0.692468,0.706355,0.686800,0.698697,0.014187,4
4,0.049196,0.004021,0.000600,4.896292e-04,friedman_mse,5,best,"{'criterion': 'friedman_mse', 'max_depth': 5, ...",0.685056,0.722808,0.692468,0.706355,0.686800,0.698697,0.014187,4



CPU times: total: 2.66 s
Wall time: 8.6 s


In [18]:
%%time
if True:
    HP_TUNNER = GridSearchCV(RandomForestRegressor(random_state=RANDOM_SEED), {'max_depth':[45, 50, 55], 'criterion' : ['squared_error'], 'n_estimators' : [200, 300]}, n_jobs=-1)
    HP_TUNNER.fit(DATA_X, DATA_Y)
    
    print('---RandomForestRegressor---')
    print(f'Best params = {HP_TUNNER.best_params_}')
    print()
    display(pd.DataFrame(HP_TUNNER.cv_results_).sort_values(by='rank_test_score', ignore_index=True).head())
    print()

---RandomForestRegressor---
Best params = {'criterion': 'squared_error', 'max_depth': 45, 'n_estimators': 300}



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,26.252842,0.817142,0.185599,0.009395,squared_error,45,300,"{'criterion': 'squared_error', 'max_depth': 45...",0.775254,0.793209,0.786368,0.797600,0.769532,0.784392,0.010587,1
1,27.023659,0.447915,0.200196,0.035138,squared_error,50,300,"{'criterion': 'squared_error', 'max_depth': 50...",0.775254,0.793200,0.786368,0.797600,0.769532,0.784391,0.010585,2
2,24.232831,0.577705,0.161601,0.020635,squared_error,55,300,"{'criterion': 'squared_error', 'max_depth': 55...",0.775254,0.793200,0.786368,0.797600,0.769532,0.784391,0.010585,2
3,17.272623,0.386849,0.123196,0.008923,squared_error,45,200,"{'criterion': 'squared_error', 'max_depth': 45...",0.773979,0.792689,0.786101,0.796931,0.770714,0.784083,0.010237,4
4,17.157229,0.852907,0.120598,0.007939,squared_error,50,200,"{'criterion': 'squared_error', 'max_depth': 50...",0.773979,0.792676,0.786101,0.796931,0.770714,0.784080,0.010235,5



CPU times: total: 19.7 s
Wall time: 1min 47s


In [15]:
%%time
if True:
    HP_TUNNER = GridSearchCV(MLPRegressor(random_state=RANDOM_SEED, max_iter=500), {'solver':['lbfgs'], 'alpha': 10.0 ** -np.arange(1, 7)}, n_jobs=-1)
    HP_TUNNER.fit(DATA_X, DATA_Y)
    
    print('---MLPRegressor---')
    print(f'Best params = {HP_TUNNER.best_params_}')
    print()
    display(pd.DataFrame(HP_TUNNER.cv_results_).sort_values(by='rank_test_score', ignore_index=True).head())
    print()

---MLPRegressor---
Best params = {'alpha': 1e-05, 'solver': 'lbfgs'}



C:\Users\mateu\git\wine-classification\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,17.353824,1.768995,0.004601,0.000490,0.00001,lbfgs,"{'alpha': 1e-05, 'solver': 'lbfgs'}",0.731259,0.748891,0.730231,0.761360,0.721513,0.738651,0.014421,1
1,17.426028,1.004810,0.004404,0.000801,0.1,lbfgs,"{'alpha': 0.1, 'solver': 'lbfgs'}",0.730507,0.748822,0.729328,0.758753,0.718337,0.737149,0.014575,2
2,19.104628,1.125378,0.004801,0.000748,0.0001,lbfgs,"{'alpha': 0.0001, 'solver': 'lbfgs'}",0.732020,0.744831,0.730464,0.757819,0.720188,0.737064,0.012999,3
3,19.676237,0.316714,0.005799,0.002233,0.001,lbfgs,"{'alpha': 0.001, 'solver': 'lbfgs'}",0.731213,0.745721,0.729767,0.757334,0.720376,0.736882,0.013051,4
4,14.186015,0.467921,0.003205,0.000979,0.000001,lbfgs,"{'alpha': 1e-06, 'solver': 'lbfgs'}",0.729147,0.747886,0.730571,0.756293,0.716317,0.736043,0.014261,5



CPU times: total: 23.9 s
Wall time: 1min 21s


In [20]:
%%time
if True:
    PIPE = Pipeline([
    ('transform', ColumnTransformer([('normalize', Normalizer(), to_use_numerical_column_numbers),
                                     ('pass', OneHotEncoder(), to_use_categorical_column_numbers)])),
    ('estimator', RandomForestRegressor(criterion='squared_error', max_depth=45, n_estimators=300, n_jobs=-1))
    ])
    PIPE.fit(DATA.to_numpy(), DATA[Y_COLUMN_NAME].to_numpy())
    joblib.dump(PIPE, '../models/price_model.joblib', compress=3)

CPU times: total: 35 s
Wall time: 7.77 s


In [21]:
CONN.close()
DB_ENGINE.dispose()